<a href="https://colab.research.google.com/github/SoumyaAIDev/Hugging_Face_LLM_models/blob/main/Alfred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("AlfredPros/smart-contracts-instructions")
print(dataset["train"].to_pandas().head(5))


README.md:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

(…)12499,20000-22999_processed.parquet.gzip:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6003 [00:00<?, ? examples/s]

                                         instruction  \
0  Make a smart contract to manage a token called...   
1  Make a smart contract to calculate the amount ...   
2  Make a smart contract to transfer and manage E...   
3  Make a smart contract to create an ERC20 token...   
4  Make a smart contract to calculate the amount ...   

                                         source_code  model  
0  pragma solidity 0.5.4;\n\ninterface IERC20 {\n...  gpt-4  
1  pragma solidity 0.6.5;\npragma experimental AB...  gpt-4  
2  pragma solidity 0.4.26;\n\ninterface IERC20 {\...  gpt-4  
3  pragma solidity 0.5.10;\n\ncontract Context {\...  gpt-4  
4  pragma solidity 0.6.5;\npragma experimental AB...  gpt-4  


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import Dataset

# Load pre-trained T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define the dataset in dictionary format
data = {
    "instruction": [
        "Make a smart contract to manage a token called 'MyToken'.",
        "Make a smart contract to calculate the amount of tokens earned over time.",
        "Make a smart contract to transfer and manage ERC20 tokens securely.",
        "Make a smart contract to create an ERC20 token with a burn feature.",
        "Make a smart contract to calculate the amount of staking rewards.",
        "Make a smart contract to manage the transfer of ownership securely.",
        "Make a smart contract to create a token with the mint and burn functionality.",
        "Make a smart contract to create a token called 'CryptoCoin'.",
        "Make a smart contract to implement a timelock for token releases.",
        "Make a smart contract to propose and vote on various DAO actions."
    ],
    "source_code": [
        "pragma solidity 0.5.4;\n\ninterface IERC20 {\n...}",  # Replace with actual code
        "pragma solidity 0.6.5;\npragma experimental ABIEncoderV2;\n...}",
        "pragma solidity 0.4.26;\n\ninterface IERC20 {\n...}",
        "pragma solidity 0.5.10;\n\ncontract Context {\n...}",
        "pragma solidity 0.6.5;\npragma experimental ABIEncoderV2;\n...}",
        "pragma solidity 0.6.12;\n\nabstract contract Context {\n...}",
        "pragma solidity 0.5.7;\n\nlibrary Roles {\n...}",
        "pragma solidity 0.5.0;\n\ncontract Context {\n...}",
        "pragma solidity 0.5.17;\n\ncontract Timelock {\n...}",
        "pragma solidity 0.7.0;\n\ninterface IOwnership {\n...}"
    ]
}

# Convert the dataset into a Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Preprocessing function
def preprocess_function(examples):
    inputs = [f"Instruction: {instr} Code:" for instr in examples["instruction"]]
    targets = examples["source_code"]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

print(tokenized_dataset)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'source_code', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})


In [ ]:

from transformers import Trainer, TrainingArguments
# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split the dataset for training and validation
train_dataset = tokenized_dataset.select(range(2))  # First two samples for training
validation_dataset = tokenized_dataset.select(range(1, 2))  # Second sample for validation

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",                   # Directory for model outputs
    evaluation_strategy="epoch",             # Evaluate at the end of each epoch
    learning_rate=2e-5,                       # Learning rate
    per_device_train_batch_size=8,           # Batch size for training
    per_device_eval_batch_size=8,            # Batch size for evaluation
    num_train_epochs=3,                      # Number of epochs
    weight_decay=0.01,                       # Weight decay for optimization
    logging_dir="./logs",                    # Directory for logs
    logging_steps=10,                        # Log every 10 steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,                              # T5 model
    args=training_args,                       # Training arguments
    train_dataset=train_dataset,              # Training dataset
    eval_dataset=validation_dataset           # Validation dataset
)

# Train the model
trainer.train()

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,24.992672
2,No log,23.982702
3,No log,23.511011


TrainOutput(global_step=3, training_loss=19.952433268229168, metrics={'train_runtime': 74.1469, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.04, 'total_flos': 812050808832.0, 'train_loss': 19.952433268229168, 'epoch': 3.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation results: {eval_results}")


Evaluation results: {'eval_loss': 23.511011123657227, 'eval_runtime': 0.0519, 'eval_samples_per_second': 19.285, 'eval_steps_per_second': 19.285, 'epoch': 3.0}


In [ ]:
from datasets import Dataset

# Define the dataset
data = {
    "instruction": [
        "Make a smart contract to manage a token called 'MyToken'.",
        "Make a smart contract to calculate the amount of tokens earned over time.",
        "Make a smart contract to transfer and manage ERC20 tokens securely.",
        "Make a smart contract to create an ERC20 token with a burn feature.",
        "Make a smart contract to calculate the amount of staking rewards.",
        "Make a smart contract to manage the transfer of ownership securely.",
        "Make a smart contract to create a token with the mint and burn functionality.",
        "Make a smart contract to create a token called 'CryptoCoin'.",
        "Make a smart contract to implement a timelock for token releases.",
        "Make a smart contract to propose and vote on various DAO actions."
    ],
    "source_code": [
        "pragma solidity 0.5.4;\n\ninterface IERC20 {\n...}",  # Replace with actual code
        "pragma solidity 0.6.5;\npragma experimental ABIEncoderV2;\n...}",
        "pragma solidity 0.4.26;\n\ninterface IERC20 {\n...}",
        "pragma solidity 0.5.10;\n\ncontract Context {\n...}",
        "pragma solidity 0.6.5;\npragma experimental ABIEncoderV2;\n...}",
        "pragma solidity 0.6.12;\n\nabstract contract Context {\n...}",
        "pragma solidity 0.5.7;\n\nlibrary Roles {\n...}",
        "pragma solidity 0.5.0;\n\ncontract Context {\n...}",
        "pragma solidity 0.5.17;\n\ncontract Timelock {\n...}",
        "pragma solidity 0.7.0;\n\ninterface IOwnership {\n...}"
    ]
}

# Convert to a Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Define a function to fetch the complete source code for a prompt
def get_source_code(dataset, prompt):
    # Filter dataset for matching instruction
    matching_entries = dataset.filter(lambda example: prompt.lower() in example["instruction"].lower())
    if len(matching_entries) == 0:
        return "No matching source code found for the given prompt."
    # Extract and return the source code
    return matching_entries["source_code"]

# User input
prompt = input("Enter a prompt: ")

# Retrieve the source code
source_codes = get_source_code(dataset, prompt)

# Display the results
if isinstance(source_codes, str):  # No match found
    print(source_codes)
else:  # Match found
    for idx, code in enumerate(source_codes):
        print(f"Matching Source Code {idx + 1}:\n{code}\n{'-' * 50}")


Enter a prompt: Make a smart contract to manage the transfer of ownership securely.


Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

Matching Source Code 1:
pragma solidity 0.6.12;

abstract contract Context {
...}
--------------------------------------------------


In [ ]:
from datasets import load_dataset
dataset = load_dataset("AlfredPros/smart-contracts-instructions")
print(dataset['train'].to_pandas().head(6003))

                                            instruction  \
0     Make a smart contract to manage a token called...   
1     Make a smart contract to calculate the amount ...   
2     Make a smart contract to transfer and manage E...   
3     Make a smart contract to create an ERC20 token...   
4     Make a smart contract to calculate the amount ...   
...                                                 ...   
5998  Make a smart contract to create a token called...   
5999  Make a smart contract to create a token called...   
6000  Make a smart contract to create an ERC20 token...   
6001  Make a smart contract to handle token transfer...   
6002  Make a smart contract to create an iterable ma...   

                                            source_code          model  
0     pragma solidity 0.5.4;\n\ninterface IERC20 {\n...          gpt-4  
1     pragma solidity 0.6.5;\npragma experimental AB...          gpt-4  
2     pragma solidity 0.4.26;\n\ninterface IERC20 {\...          gpt-4  

In [ ]:

import pandas as pd

# Complete dataset example (replace with the full dataset as needed)
data = {
    "instruction".extend[
        "Make a smart contract to manage a token called 'MyToken'.",
        "Make a smart contract to calculate the amount of tokens earned over time.",
        "Make a smart contract to transfer and manage ERC20 tokens securely.",
        "Make a smart contract to create an ERC20 token with a burn feature.",
        "Make a smart contract to calculate the amount of staking rewards.",
        "Make a smart contract to create a token called 'CryptoCoin'.",
        "Make a smart contract to propose and vote on various DAO actions.",
        "Make a smart contract to implement a timelock for token releases.",
        "Make a smart contract to manage the transfer of ownership securely.",
        "Make a smart contract to create a token with the mint and burn functionality.",

    ],
    "source_code".extend [
        "pragma solidity 0.5.4;\n\ninterface IERC20 {\nfunction totalSupply() external view returns (uint256);}",
        "pragma solidity 0.6.5;\npragma experimental ABIEncoderV2;\ncontract RewardCalculator { function calculateRewards() public pure returns (uint256) { return 100; }}",
        "pragma solidity 0.4.26;\n\ninterface IERC20 {\nfunction transfer(address recipient, uint256 amount) external returns (bool);}",
        "pragma solidity 0.5.10;\n\ncontract BurnableToken {\nfunction burn(uint256 amount) public { /* Burn logic */ }}",
        "pragma solidity 0.6.5;\npragma experimental ABIEncoderV2;\ncontract StakingRewards { function getStakingRewards() public view returns (uint256) { return 50; }}",
        "pragma solidity 0.5.0;\ncontract CryptoCoin { string public name = 'CryptoCoin'; }",
        "pragma solidity 0.7.0;\ncontract DAOProposal { function propose(string memory proposal) public { /* Proposal logic */ }}",
        "pragma solidity 0.5.17;\ncontract Timelock { function releaseTokens() public { /* Release logic */ }}",
  ],
    data["model"].extend(["gpt-4", "gpt-4", "gpt-4"])", "gpt-4", "gpt-4", "gpt-4"],
}

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Define a function to search the dataset for a prompt
def search_and_execute(dataframe, prompt):
    """
    Searches the dataset for instructions matching the prompt and retrieves the corresponding source_code.

    Args:
    - dataframe: pd.DataFrame containing the dataset.
    - prompt: str, the input prompt to search for.

    Returns:
    - A DataFrame with matching instruction and source_code or a message if no matches are found.
    """
    # Filter rows where the instruction contains the prompt (case-insensitive)
    filtered_rows = dataframe[dataframe["instruction"].str.contains(prompt, case=False, na=False)]

    if filtered_rows.empty:
        return "No matching source code found for the given prompt."

    # Return the matching rows
    return filtered_rows

# Prompt input from the user
prompt = input("Enter a prompt to search in the dataset: ")

# Execute the search and retrieve results
results = search_and_execute(df, prompt)

# Display the results
if isinstance(results, str):  # If no matches found
    print(results)
else:  # Display the matching rows
    for idx, row in results.iterrows():
        print(f"Instruction: {row['instruction']}")
        print(f"Source Code:\n{row['source_code']}")
        print("-" * 50)


SyntaxError: unterminated string literal (detected at line 28) (<ipython-input-4-00825f336759>, line 28)

import pandas as pd

# Complete dataset example (replace with the full dataset as needed)
data["instruction"].extend([
    "Make a smart contract to mint NFTs securely.",
    "Make a smart contract to handle multi-signature wallet transactions.",
    "Make a smart contract to auction digital assets with a time limit.",
])

data["source_code"].extend([
    "pragma solidity 0.8.0;\n\ncontract NFTMinter {\nfunction mint() public { /* Minting logic */ }}",
    "pragma solidity 0.8.0;\n\ncontract MultiSigWallet {\nfunction execute() public { /* Multi-Sig logic */ }}",
    "pragma solidity 0.8.0;\n\ncontract Auction {\nfunction bid() public { /* Auction logic */ }}",
])



# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Define a function to search the dataset for a prompt
def search_and_execute(dataframe, prompt):
    """
    Searches the dataset for instructions matching the prompt and retrieves the corresponding source_code.
    
    Args:
    - dataframe: pd.DataFrame containing the dataset.
    - prompt: str, the input prompt to search for.

    Returns:
    - A DataFrame with matching instruction and source_code or a message if no matches are found.
    """
    # Filter rows where the instruction contains the prompt (case-insensitive)
    filtered_rows = dataframe[dataframe["instruction"].str.contains(prompt, case=False, na=False)]
    
    if filtered_rows.empty:
        return "No matching source code found for the given prompt."
    
    # Return the matching rows
    return filtered_rows

# Prompt input from the user
prompt = input("Enter a prompt to search in the dataset: ")

# Execute the search and retrieve results
results = search_and_execute(df, prompt)

# Display the results
if isinstance(results, str):  # If no matches found
    print(results)
else:  # Display the matching rows
    for idx, row in results.iterrows():
        print(f"Instruction: {row['instruction']}")
        print(f"Source Code:\n{row['source_code']}")
        print("-" * 50)


In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("AlfredPros/smart-contracts-instructions")

# Define a function to search for matching instructions and display source code
def search_dataset(dataset, prompt):
    """
    Searches the dataset for instructions matching the prompt and retrieves the corresponding source code.

    Args:
    - dataset: The Hugging Face dataset.
    - prompt: The user-provided search query.

    Returns:
    - Matching rows (instruction and source_code) or a message if no matches are found.
    """
    # Filter rows where the instruction contains the prompt (case-insensitive)
    filtered_rows = dataset["train"].filter(lambda example: prompt.lower() in example["instruction"].lower())

    if len(filtered_rows) == 0:
        return "No matching entries found for the given prompt."

    # Return matching instructions and source codes
    return filtered_rows

# Prompt input from the user
prompt = input("Enter a prompt to search in the dataset: ")

# Execute the search
results = search_dataset(dataset, prompt)

# Display results
if isinstance(results, str):  # If no matches found
    print(results)
else:
    for idx, row in enumerate(results):
        print(f"Instruction: {row['instruction']}")
        print(f"Source Code:\n{row['source_code']}")
        print("-" * 50)


README.md:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

(…)12499,20000-22999_processed.parquet.gzip:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6003 [00:00<?, ? examples/s]

In [ ]:
!pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


# Add more rows to the data dictionary
data["instruction"].extend([
    "Make a smart contract to mint NFTs securely.",
    "Make a smart contract to handle multi-signature wallet transactions.",
    "Make a smart contract to auction digital assets with a time limit.",
])

data["source_code"].extend([
    "pragma solidity 0.8.0;\n\ncontract NFTMinter {\nfunction mint() public { /* Minting logic */ }}",
    "pragma solidity 0.8.0;\n\ncontract MultiSigWallet {\nfunction execute() public { /* Multi-Sig logic */ }}",
    "pragma solidity 0.8.0;\n\ncontract Auction {\nfunction bid() public { /* Auction logic */ }}",
])

data["model"].extend(["gpt-4", "gpt-4", "gpt-4"])
